In [ ]:
# This is from the Udacity youtube tutorial links below
# https://www.youtube.com/watch?v=rpxZ87YFg0M&feature=youtu.be

import csv
import cv2 
import numpy as np

with open('data\driving_log.csv') as csvfile:
   
    lines = [] 

    reader = csv.reader(csvfile)
    for line in reader:
        lines.append(line)

        
#going to get rid of troublesome firtst CSV line here
lines.pop(0)

images = []
measurements = []
        
for line in (lines):
    for i in range(2):#was 3
    
        source_path = line[i]
        tokens = source_path.split('/')
        filename = tokens[-1]
        #local_path = "/data/IMG/" + filename #+'.jpg'
        local_path = "C:/Users/Admin/Desktop/SELF DRIVING CAR ND/Project1/CarND-Term1-Starter-Kit/CarND-Behavioral-Cloning-P3/data/IMG/" + filename
        image = cv2.imread(local_path)
        #image = image[70:135, 0:320]#hoping this crop here will work
        
        images.append(image)
    
    correction = 0.2
    #measurement =  float(line[3]) #worked for teacher??
    measurement =  float(line[3])
    measurements.append(measurement)
    measurements.append(measurement + correction)
   # measurements.append(measurement - correction)

print(len(images))#center_2016_12_01_13_31_13_786
print(len(measurements))
#print(images[1])
#print(measurements[0])

#images.pop(0)#first thing in csv file was not data top of column description
#measurements.pop(0)

augmented_images = []
augmented_measurements = []

for image,measurement in zip( images,measurements):
    augmented_images.append(image)
    augmented_measurements.append(measurement)
    flipped_image = cv2.flip(image,1)
    flipped_measurement = float(measurement) * -1.0
    augmented_images.append(flipped_image)
    augmented_measurements.append(flipped_measurement)
   #print(measurement)
   # exit()
    

#x_train = np.array(images)  #before augmentation and origianl LeNet
#y_train = np.array(measurements)

x_train = np.array(augmented_images)  #before augmentation and origianl LeNet
y_train = np.array(augmented_measurements)

print(x_train.shape)
print(y_train.shape)

import keras
from keras.models import Sequential
from keras.layers import Flatten,Dense,Lambda
from keras.layers.convolutional import Convolution2D , Cropping2D
from keras.layers.pooling import MaxPooling2D

#ValueError: Error when checking model input: expected lambda_input_1
#to have shape (None, 160, 320, 3) but got array with shape (48216, 65, 320, 3)

#maybe try grayscale?
model = Sequential()
model.add(Lambda(lambda x: x/255.0 -0.5,input_shape = (160,320,3))) #out for cropping else where
#model.add(Lambda(lambda x: x/255.0 -0.5,input_shape = (65,320,3))) 
#model.add(Flatten(input_shape = (160,320,3))) #first attempt beofre lambda, trying without nromailisation again attempt 6
model.add(Cropping2D(cropping=((70,25),(0,0)))) #trying to crop at start to speed up
model.add(Convolution2D(6,5,5,activation='relu'))#Lenet
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(120))#Lenet
model.add(Dense(84))#Lenet
model.add(Dense(1))

model.compile(optimizer = 'Adam', loss = 'mse')
model.fit(x_train,y_train , validation_split= 0.2, shuffle = True,nb_epoch=3)

#model.save('model.h5') #change this for each time#just went hard right circles
#model.save('model2.h5') # slightly better 99% hard right circles think I overwrote this
#model.save('model3.h5')  # LeNet goes around the track but touches edge twice possibly good enough
#model.save('model4.h5')# LENEt without cropping but with flip
#model.save('model5.h5')#flipped and cropped and epoch 3 got passed brown corner but not better than very firt LENet
#model.save('model6.h5')# this was zero center no normalisation myuch worse
#model.save('model7.h5')# doing nothing to values no normail no zero center kinda funny goes off road steers left and right putting normailisation back in
model.save('model8.h5')#this has left camera and middle camera because 3 camera too big worse than LEnet first attmept does OK but goes right and into water before bridge
model.save('model9.h5')